In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error
import optuna
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost.callback import EarlyStopping

/Users/nataschajademinnitt/mamba/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Feature engineering

**Lag Features** 
Prices today often depend on prices in the past — the model can’t infer that automatically from timestamps, so we explicitly provide recent values.

* lag 1 = 1 month
* lag 3 = 3 months
* lag 6 = 6 months

**Rolling statistics**
Beyond single past points, we want the model to know recent trend and volatility.
* roll_mean_3 = mean of previous 3 months
* roll_std_3 = standard deviation of previous 3 months
* roll_mean_6 = mean of previous 6 months

**Seasonal encoding**
This gives two coordinates on the unit circle — January and December are now close again. Models can then learn patterns like “prices rise around harvest season” continuously across years.



In [2]:
# Load data
df = pd.read_parquet("/Users/nataschajademinnitt/Documents/5_data/food_security/etl/ethiopia_foodprices_model_panel.parquet")

In [3]:
# ------------------------------------------------------------------------------
# BUILD FORECASTING FEATURES
# ------------------------------------------------------------------------------

def add_missing_flags(df, cols):
    for c in cols:
        df[f"{c}_was_missing"] = df[c].isna().astype("int8")
    return df

def national_same_month_fill(df, col):
    # median over all regions/products at the same month
    same_mo = df.groupby("month")[col].transform("median")
    df[col] = df[col].fillna(same_mo)
    return df

def past_only_roll_fill(df, key_cols, col, window=3):
    # per group, use past-only rolling mean as a gentle fallback
    df = df.sort_values(key_cols + ["month"])
    def _roll(g):
        s = g[col]
        s_fallback = (
            s.shift(1)  # ensure past-only
             .rolling(window=window, min_periods=1)
             .mean()
        )
        return s.fillna(s_fallback)
    df[col] = df.groupby(key_cols, group_keys=False)[col].apply(_roll)
    return df

def ensure_cols(df: pd.DataFrame, cols: list[str], dtype=float) -> pd.DataFrame:
    """Add any missing columns with NaN so downstream code is safe."""
    for c in cols:
        if c not in df.columns:
            df[c] = np.nan
        if dtype is not None and c in df.columns:
            try:
                df[c] = df[c].astype(dtype)
            except Exception:
                pass
    return df

def month_start_series(s):
    return pd.to_datetime(s, errors="coerce").dt.to_period("M").dt.to_timestamp()


def impute_features_for_forecasting(df_in: pd.DataFrame) -> pd.DataFrame:
    df = df_in.copy()
    if "month" in df.columns:
        df["month"] = month_start_series(df["month"])

    # --- Ensure optional columns exist so code never KeyErrors
    rain_cols = ["rfh_month", "rfh_avg_month", "rfq_month", "rain_anom_pct"]
    df = ensure_cols(df, rain_cols, dtype="float64")

    fao_cols = ["fao_category_index", "fao_food_price_index"]
    df = ensure_cols(df, fao_cols, dtype="float64")

    if "ptm_severity" not in df.columns:
        df["ptm_severity"] = np.nan
    if "population_2023" not in df.columns:
        df["population_2023"] = np.nan

    # --- Missingness flags BEFORE filling
    for c in rain_cols:
        df[f"{c}_was_missing"] = df[c].isna().astype("int8")

    # --- Rain: recompute anomaly where possible
    can_compute = df["rfh_month"].notna() & df["rfh_avg_month"].gt(0)
    df.loc[can_compute, "rain_anom_pct"] = 100.0 * (
        df.loc[can_compute, "rfh_month"] / df.loc[can_compute, "rfh_avg_month"] - 1.0
    )

    # Same-month “national” median (over all rows in that month)
    def fill_same_month(col):
        df[col] = df[col].fillna(df.groupby("month")[col].transform("median"))
    for c in ["rfh_month", "rfh_avg_month", "rfq_month", "rain_anom_pct"]:
        if c in df.columns:
            fill_same_month(c)

    # Past-only rolling fallback per admin_1
    def past_roll(col, window=3):
        df.sort_values(["admin_1", "month"], inplace=True)
        df[col] = (df.groupby("admin_1", group_keys=False)[col]
                     .apply(lambda s: s.fillna(s.shift(1).rolling(window, min_periods=1).mean())))
    for c in ["rfh_month", "rfh_avg_month", "rfq_month", "rain_anom_pct"]:
        if c in df.columns:
            past_roll(c, window=3)

    # --- GMM: ptm_severity (ordinal with many gaps) + flag
    df["ptm_missing"] = df["ptm_severity"].isna().astype("int8")
    df["ptm_severity"] = df["ptm_severity"].fillna(0).astype("int8")

    # --- Population: static per admin
    df["pop_missing"] = df["population_2023"].isna().astype("int8")
    admin_med = df.groupby("admin_1")["population_2023"].transform("median")
    df["population_2023"] = df["population_2023"].fillna(admin_med)
    df["population_2023"] = df["population_2023"].fillna(df["population_2023"].median())

    # --- FAO indices: same-month fill, then global ffill by time
    for c in fao_cols:
        df[c] = df[c].fillna(df.groupby("month")[c].transform("median"))
    df.sort_values(["month"], inplace=True)
    for c in fao_cols:
        df[c] = df[c].ffill()

    return df


def build_forecast_matrix(df):
    """
    df: your tidy panel with value_imputed and exogenous features (after imputation).
    Returns df_model with lags/rolling stats & calendar features.
    """
    df = df.copy()
    df = df.sort_values(["admin_1","product","month"])

    # Lags & rolling (past-only)
    grp = df.groupby(["admin_1","product"], group_keys=False)
    df["y"] = df["value_imputed"]
    df["y_lag1"] = grp["y"].shift(1)
    df["y_lag3"] = grp["y"].shift(3)
    df["y_lag6"] = grp["y"].shift(6)

    df["roll_mean_3"] = grp["y"].apply(lambda s: s.shift(1).rolling(3, min_periods=1).mean())
    df["roll_std_3"]  = grp["y"].apply(lambda s: s.shift(1).rolling(3, min_periods=2).std())
    df["roll_mean_6"] = grp["y"].apply(lambda s: s.shift(1).rolling(6, min_periods=2).mean())

    # Calendar features
    df["month_num"] = df["month"].dt.month
    # Optional seasonal encoding
    df["mo_sin"] = np.sin(2*np.pi*df["month_num"]/12)
    df["mo_cos"] = np.cos(2*np.pi*df["month_num"]/12)

    # Drop rows that don’t have minimal lags (optional, e.g., first 6 months per series)
    df = df[df["y_lag1"].notna()].copy()

    return df

df_feat = impute_features_for_forecasting(df)
df_model = build_forecast_matrix(df_feat)

# Modelling

In [4]:
# ========= Helpers =========
def smape(y_true, y_pred, eps=1e-9):
    denom = (np.abs(y_true) + np.abs(y_pred) + eps) / 2.0
    return np.mean(np.abs(y_pred - y_true) / denom) * 100.0

def pick_features(df: pd.DataFrame):
    # Columns to exclude from X (target, keys, and QC-only fields)
    drop = {
        "y", "value_imputed", "value_median", "value_mean", "value_orig",
        "n_obs", "impute_method", "sources",
        "month", "period_date",  # keys / time
    }
    # Add any columns that are not numeric (we’ll encode 2 categoricals below)
    non_num = set(df.columns[df.dtypes == "object"])
    # We'll keep admin_1 & product after encoding; drop other object cols if present
    drop |= (non_num - {"admin_1", "product"})
    feats = [c for c in df.columns if c not in drop]
    return feats

def encode_categoricals(df: pd.DataFrame):
    """Ordinal-encode admin_1 and product to avoid huge one-hot matrices.
    For many categories, this is compact & fast; models like XGB handle it well.
    """
    out = df.copy()
    for col in ["admin_1", "product"]:
        if col in out.columns:
            out[col] = out[col].astype("category")
            out[f"{col}_code"] = out[col].cat.codes.astype("int32")
    return out

def time_based_split(df: pd.DataFrame, test_horizon_months: int = 6):
    """Hold out the last H months as test set (by calendar), keeping all series."""
    last_month = df["month"].max().to_period("M").to_timestamp()
    cut_month  = (last_month.to_period("M") - test_horizon_months).to_timestamp()
    train = df[df["month"] <= cut_month].copy()
    test  = df[df["month"] >  cut_month].copy()
    return train, test

def get_X_y(df: pd.DataFrame, feature_cols: list[str]):
    X = df[feature_cols].copy()
    y = df["y"].astype("float32").to_numpy()
    return X, y

def eval_report(y_true, y_pred, label=""):
    mae  = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    s    = smape(y_true, y_pred)
    print(f"{label} | MAE: {mae:,.3f}  RMSE: {rmse:,.3f}  sMAPE: {s:,.2f}%")
    return {"mae": mae, "rmse": rmse, "smape": s}


In [ ]:
def fit_xgb_compat(params, X_tr, y_tr, X_val=None, y_val=None, early_rounds=100):
    """
    Tries XGBoost >=2.0 API first (callbacks+eval_metric in params).
    Falls back to <2.0 API (eval_metric + early_stopping_rounds in fit).
    Last fallback: no early stopping.
    """
    # 1) Try v2-style: callbacks + eval_metric in params
    try:
        from xgboost.callback import EarlyStopping  # only present in newer versions
        model = XGBRegressor(**params)
        if X_val is not None:
            model.fit(
                X_tr, y_tr,
                eval_set=[(X_val, y_val)],
                callbacks=[EarlyStopping(rounds=early_rounds, save_best=True)]
            )
        else:
            model.fit(X_tr, y_tr)
        return model
    except TypeError:
        pass  # fall through to older API
    except ImportError:
        pass

    # 2) Try v1-style: eval_metric + early_stopping_rounds in fit
    try:
        model = XGBRegressor(**{k:v for k,v in params.items() if k != "eval_metric"})
        if X_val is not None:
            model.fit(
                X_tr, y_tr,
                eval_set=[(X_val, y_val)],
                eval_metric=params.get("eval_metric", "rmse"),
                early_stopping_rounds=early_rounds,
            )
        else:
            model.fit(X_tr, y_tr)
        return model
    except TypeError:
        pass

    # 3) Last resort: just fit without early stopping
    model = XGBRegressor(**{k:v for k,v in params.items() if k != "eval_metric"})
    model.fit(X_tr, y_tr)
    return model

In [5]:
# ========= Prepare data =========
# df_model should already exist: keys: admin_1, product, month; target: y
assert {"admin_1", "product", "month", "y"}.issubset(df_model.columns)

# Encode categoricals and keep the encoded columns as features
dfX = encode_categoricals(df_model)
feature_cols = pick_features(dfX)

# Make sure encoded cols are included
for col in ["admin_1_code", "product_code"]:
    if col in dfX.columns and col not in feature_cols:
        feature_cols.append(col)

# Optional: ensure all features are numeric
dfX[feature_cols] = dfX[feature_cols].apply(pd.to_numeric, errors="coerce")

# ========= Train/Test split (forecast holdout) =========
HORIZON = 6  # months
train_df, test_df = time_based_split(dfX, test_horizon_months=HORIZON)
X_train, y_train = get_X_y(train_df, feature_cols)
X_test,  y_test  = get_X_y(test_df,  feature_cols)

## Baseline

## Optimized – Global

In [ ]:
# ========= Optuna objective with 5-fold TimeSeriesSplit =========

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 300, 2000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_child_weight": trial.suggest_float("min_child_weight", 1e-2, 10.0, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "tree_method": "hist",
        "random_state": 42,
        "n_jobs": 0,
        # safe to include; ignored by the fallback path if needed
        "eval_metric": "rmse",
    }

    tscv = TimeSeriesSplit(n_splits=5)
    smapes = []

    for tr_idx, va_idx in tscv.split(X_train):
        X_tr, X_val = X_train.iloc[tr_idx], X_train.iloc[va_idx]
        y_tr, y_val = y_train[tr_idx],    y_train[va_idx]

        model = fit_xgb_compat(params, X_tr, y_tr, X_val, y_val, early_rounds=100)
        yhat = model.predict(X_val)

        # your sMAPE
        denom = (np.abs(y_val) + np.abs(yhat) + 1e-9) / 2.0
        s = np.mean(np.abs(yhat - y_val) / denom) * 100.0
        smapes.append(s)

        trial.report(float(np.mean(smapes)), step=len(smapes))
        if trial.should_prune():
            import optuna
            raise optuna.exceptions.TrialPruned()

    return float(np.mean(smapes))


# ========= Run study =========
study = optuna.create_study(direction="minimize", study_name="xgb-forecast-smape")
study.optimize(objective, n_trials=50, show_progress_bar=False)  # adjust trials

print("Best sMAPE:", study.best_value)
print("Best params:", study.best_trial.params)

[I 2025-10-25 10:27:56,450] A new study created in memory with name: xgb-forecast-smape
[I 2025-10-25 10:28:02,203] Trial 0 finished with value: 38.96247100830078 and parameters: {'n_estimators': 1714, 'learning_rate': 0.014980442194867364, 'max_depth': 3, 'min_child_weight': 0.04118537632553031, 'subsample': 0.5296816210385351, 'colsample_bytree': 0.7863047583700602, 'gamma': 2.340408897143101, 'reg_alpha': 1.4877487247850677e-07, 'reg_lambda': 0.00039287435670747837}. Best is trial 0 with value: 38.96247100830078.
[I 2025-10-25 10:28:10,269] Trial 1 finished with value: 31.506664276123047 and parameters: {'n_estimators': 1519, 'learning_rate': 0.01259762168851857, 'max_depth': 4, 'min_child_weight': 0.40492333599412705, 'subsample': 0.7866397346298584, 'colsample_bytree': 0.9427179039843608, 'gamma': 3.5700272128073367, 'reg_alpha': 0.0004994658493833769, 'reg_lambda': 6.484485569090389}. Best is trial 1 with value: 31.506664276123047.
[I 2025-10-25 10:28:13,328] Trial 2 finished wit

Best sMAPE: 9.015694618225098
Best params: {'n_estimators': 1635, 'learning_rate': 0.011357830666199348, 'max_depth': 10, 'min_child_weight': 0.953558564308704, 'subsample': 0.9128347562586963, 'colsample_bytree': 0.8953157826629276, 'gamma': 3.499555969891328, 'reg_alpha': 0.012521736818631172, 'reg_lambda': 0.003049668143793192}


In [9]:
# Fit with the compat wrapper from earlier
final_model = fit_xgb_compat(best_params, X_train, y_train, X_val=X_test, y_val=y_test, early_rounds=200)
y_pred_test = final_model.predict(X_test)

mae  = mean_absolute_error(y_test, y_pred_test)
rmse = root_mean_squared_error(y_test, y_pred_test)
smape_val = np.mean(np.abs(y_pred_test - y_test) / ((np.abs(y_test)+np.abs(y_pred_test)+1e-9)/2)) * 100
print(f"TEST | MAE: {mae:,.3f}  RMSE: {rmse:,.3f}  sMAPE: {smape_val:,.2f}%")

# Attach predictions back to rows
test_out = test_df.copy()
test_out["y_pred"] = y_pred_test


TEST | MAE: 1,710.771  RMSE: 7,300.389  sMAPE: 11.66%


## Optimized – Modelling per product

In [ ]:
# ===================== STAPLES-ONLY PRICE FORECAST PIPELINE =====================
import numpy as np
import pandas as pd
from pathlib import Path
import warnings

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
import optuna
from xgboost import XGBRegressor

# ---- config ----
PARQUET_PATH = "/Users/nataschajademinnitt/Documents/5_data/food_security/etl/ethiopia_foodprices_model_panel.parquet"
TEST_HORIZON_MONTHS = 6      # last H calendar months = test
N_TRIALS = 40                # Optuna trials (adjust up for more thorough search)

# ---- helpers ----
def month_start(s: pd.Series) -> pd.Series:
    return pd.to_datetime(s, errors="coerce").dt.to_period("M").dt.to_timestamp()

def smape(y_true, y_pred, eps=1e-9) -> float:
    y_true = np.asarray(y_true, float)
    y_pred = np.asarray(y_pred, float)
    denom = (np.abs(y_true) + np.abs(y_pred) + eps) / 2.0
    return float(np.mean(np.abs(y_pred - y_true) / denom) * 100.0)

def rmse(y_true, y_pred) -> float:
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def ensure_cols(df: pd.DataFrame, cols: list[str], dtype="float64") -> pd.DataFrame:
    out = df.copy()
    for c in cols:
        if c not in out.columns:
            out[c] = np.nan
        if dtype is not None:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                try:
                    out[c] = out[c].astype(dtype)
                except Exception:
                    pass
    return out

def fit_xgb_compat(params, X_tr, y_tr, X_val=None, y_val=None, early_rounds=100):
    """Try XGBoost >=2 callbacks; fall back to <2.0 API; last resort w/o early stopping."""
    # Try newer API
    try:
        from xgboost.callback import EarlyStopping
        mdl = XGBRegressor(**params)
        if X_val is not None:
            mdl.fit(X_tr, y_tr, eval_set=[(X_val, y_val)],
                    callbacks=[EarlyStopping(rounds=early_rounds, save_best=True)])
        else:
            mdl.fit(X_tr, y_tr)
        return mdl
    except (TypeError, ImportError):
        pass
    # Fallback: older API
    try:
        mdl = XGBRegressor(**{k: v for k, v in params.items() if k != "eval_metric"})
        if X_val is not None:
            mdl.fit(X_tr, y_tr, eval_set=[(X_val, y_val)],
                    eval_metric=params.get("eval_metric", "rmse"),
                    early_stopping_rounds=early_rounds)
        else:
            mdl.fit(X_tr, y_tr)
        return mdl
    except TypeError:
        pass
    # Last resort
    mdl = XGBRegressor(**{k: v for k, v in params.items() if k != "eval_metric"})
    mdl.fit(X_tr, y_tr)
    return mdl

# ---- de-categorize numerics (prevents 'new category' errors) ----
KEY_CATS = {"admin_1", "product"}
NUMERIC_LIKELY = {
    "value_imputed","value_mean","value_median","value_orig","n_obs",
    "rfh_month","rfh_avg_month","rfq_month","rain_anom_pct",
    "fao_category_index","fao_food_price_index",
    "ptm_severity","population_2023",
    "y","y_lag1","y_lag3","y_lag6","y_lag12",
    "roll_mean_3","roll_std_3","roll_mean_12",
    "month_num","mo_sin","mo_cos",
}

def decategorize_numerics(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in out.columns:
        if c in KEY_CATS:
            # keep IDs as category
            if not pd.api.types.is_categorical_dtype(out[c]):
                out[c] = out[c].astype("category")
            continue
        if (c in NUMERIC_LIKELY or
            pd.api.types.is_categorical_dtype(out[c]) or
            pd.api.types.is_object_dtype(out[c])):
            out[c] = pd.to_numeric(out[c], errors="coerce")
    return out

# ---- feature imputation for optional exogenous vars ----
def impute_features(df_in: pd.DataFrame) -> pd.DataFrame:
    df = decategorize_numerics(df_in)
    df["month"] = month_start(df["month"])

    rain_cols = ["rfh_month", "rfh_avg_month", "rfq_month", "rain_anom_pct"]
    fao_cols  = ["fao_category_index", "fao_food_price_index"]
    df = ensure_cols(df, rain_cols + fao_cols)

    # missingness flags first
    for c in rain_cols:
        df[f"{c}_was_missing"] = df[c].isna().astype("int8")

    # recompute anomaly where possible
    can = df["rfh_month"].notna() & df["rfh_avg_month"].gt(0)
    df.loc[can, "rain_anom_pct"] = 100.0 * (df.loc[can, "rfh_month"] / df.loc[can, "rfh_avg_month"] - 1.0)

    # same-month median fill
    for c in rain_cols + fao_cols:
        df[c] = df[c].fillna(df.groupby("month", observed=False)[c].transform("median"))

    # gentle past-only rolling fill by admin
    df = df.sort_values(["admin_1", "month"])
    for c in rain_cols:
        df[c] = (df.groupby("admin_1", group_keys=False)[c]
                   .apply(lambda s: s.fillna(s.shift(1).rolling(3, min_periods=1).mean())))

    # GMM & population (if present)
    if "ptm_severity" in df.columns:
        df["ptm_missing"] = df["ptm_severity"].isna().astype("int8")
        df["ptm_severity"] = df["ptm_severity"].fillna(0).astype("int8")
    else:
        df["ptm_missing"] = 1; df["ptm_severity"] = 0

    if "population_2023" in df.columns:
        df["pop_missing"] = df["population_2023"].isna().astype("int8")
        adm_med = df.groupby("admin_1", observed=False)["population_2023"].transform("median")
        df["population_2023"] = df["population_2023"].fillna(adm_med)
        df["population_2023"] = df["population_2023"].fillna(df["population_2023"].median())
    else:
        df["pop_missing"] = 1; df["population_2023"] = np.nan

    # FAO forward-fill by time
    df = df.sort_values(["month"])
    for c in fao_cols:
        df[c] = df[c].ffill()

    return df

# ---- build lag/roll + seasonal features (per admin_1, product) ----
def build_features(df_in: pd.DataFrame) -> pd.DataFrame:
    df = df_in.sort_values(["admin_1", "product", "month"]).copy()
    df["y"] = df["value_imputed"].astype("float32")

    g = df.groupby(["admin_1", "product"], group_keys=False)["y"]
    df["y_lag1"]  = g.shift(1)
    df["y_lag3"]  = g.shift(3)
    df["y_lag6"]  = g.shift(6)
    df["y_lag12"] = g.shift(12)

    df["roll_mean_3"]  = g.apply(lambda s: s.shift(1).rolling(3,  min_periods=1).mean())
    df["roll_std_3"]   = g.apply(lambda s: s.shift(1).rolling(3,  min_periods=2).std())
    df["roll_mean_12"] = g.apply(lambda s: s.shift(1).rolling(12, min_periods=3).mean())

    df["month_num"] = df["month"].dt.month
    df["mo_sin"] = np.sin(2*np.pi*df["month_num"]/12.0)
    df["mo_cos"] = np.cos(2*np.pi*df["month_num"]/12.0)

    # require minimal history
    df = df[df["y_lag1"].notna()].copy()
    return df

def encode_ids(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in ["admin_1", "product"]:
        out[c] = out[c].astype("category")
        out[f"{c}_code"] = out[c].cat.codes.astype("int32")
    return out

def pick_features(df: pd.DataFrame) -> list[str]:
    drop = {
        "y", "month", "admin_1", "product",
        "value_imputed", "value_median", "value_mean", "value_orig",
        "n_obs", "impute_method", "sources", "period_date"
    }
    return [c for c in df.columns if c not in drop]

def time_split(df: pd.DataFrame, horizon=6):
    df = df.sort_values(["month", "admin_1", "product"]).reset_index(drop=True)
    last_m = df["month"].max().to_period("M").to_timestamp()
    cut_m  = (last_m.to_period("M") - horizon).to_timestamp()
    return df[df["month"] <= cut_m].copy(), df[df["month"] > cut_m].copy()

# ---- Optuna tuning (time-aware CV; assumes X_train already time-sorted) ----
def tune_xgb(X_train, y_train, n_trials=40, seed=42):
    tscv = TimeSeriesSplit(n_splits=5)

    def objective(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 300, 2000),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "min_child_weight": trial.suggest_float("min_child_weight", 1e-2, 10.0, log=True),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "gamma": trial.suggest_float("gamma", 0.0, 5.0),
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
            "tree_method": "hist", "random_state": seed, "n_jobs": -1, "eval_metric": "rmse",
        }
        smapes = []
        for tr_idx, va_idx in tscv.split(X_train):
            X_tr, X_val = X_train.iloc[tr_idx], X_train.iloc[va_idx]
            y_tr, y_val = y_train[tr_idx],    y_train[va_idx]
            mdl = fit_xgb_compat(params, X_tr, y_tr, X_val, y_val, early_rounds=100)
            smapes.append(smape(y_val, mdl.predict(X_val)))
            trial.report(float(np.mean(smapes)), step=len(smapes))
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()
        return float(np.mean(smapes))

    study = optuna.create_study(direction="minimize", study_name="xgb-staples-smape")
    study.optimize(objective, n_trials=n_trials, show_progress_bar=False)
    print("Best sMAPE:", study.best_value)
    print("Best params:", study.best_trial.params)
    best = study.best_trial.params
    best.update({"tree_method": "hist", "random_state": seed, "n_jobs": -1, "eval_metric": "rmse"})
    return best

# ============================== RUN STAPLES ONLY ===============================
# 1) Load
panel = pd.read_parquet(PARQUET_PATH)
assert {"admin_1", "product", "month", "value_imputed"}.issubset(panel.columns)

# De-categorize numerics once after load
panel = decategorize_numerics(panel)
# Force any stray non-key categoricals to numeric, just in case
suspect_cats = [c for c in panel.columns
                if pd.api.types.is_categorical_dtype(panel[c]) and c not in KEY_CATS]
for c in suspect_cats:
    panel[c] = pd.to_numeric(panel[c].astype(object), errors="coerce")

# 2) Drop livestock
LIVESTOCK = {
    "Goats (Local Quality)", "Sheep (Local Quality)",
    "Camels (Local Quality)", "Oxen (Local Quality)"
}
staples = panel[~panel["product"].isin(LIVESTOCK)].copy()

# 3) Feature imputations & matrix
staples = impute_features(staples)
df_feat  = build_features(staples)
df_feat  = encode_ids(df_feat)

# 4) Split & features
df_feat = df_feat.sort_values(["month", "admin_1", "product"]).reset_index(drop=True)
feats = pick_features(df_feat)
df_feat[feats] = df_feat[feats].apply(pd.to_numeric, errors="coerce").astype("float32")

train_df, test_df = time_split(df_feat, horizon=TEST_HORIZON_MONTHS)
X_train, X_test = train_df[feats], test_df[feats]

# --- log target
y_train_log = np.log1p(train_df["y"].to_numpy())
y_test_true = test_df["y"].to_numpy()

# 5) Tune on TRAIN
print(f"Tuning staples model with {N_TRIALS} trials…")
best_params = tune_xgb(X_train, y_train_log, n_trials=N_TRIALS, seed=42)

# 6) Early stopping on tail of TRAIN (no leakage)
last_train_m = train_df["month"].max()
val_cut = (last_train_m.to_period("M") - 2).to_timestamp()  # last 3 months as ES
val_mask = train_df["month"] >= val_cut
X_tr_es, y_tr_es = X_train[~val_mask], y_train_log[~val_mask]
X_va_es, y_va_es = X_train[val_mask],  y_train_log[val_mask]
if len(X_va_es) == 0 or len(X_tr_es) == 0:
    X_tr_es, y_tr_es = X_train, y_train_log
    X_va_es = y_va_es = None

# 7) Fit & predict (invert log)
final_model = fit_xgb_compat(best_params, X_tr_es, y_tr_es, X_va_es, y_va_es, early_rounds=200)
y_pred_test = np.expm1(final_model.predict(X_test))

# 8) Light, product-wise caps (optional but robust)
lo = (train_df.groupby("product", observed=False)["y"].quantile(0.01))
hi = (train_df.groupby("product", observed=False)["y"].quantile(0.99))
m = test_df["product"].map(lo).fillna(train_df["y"].quantile(0.01)) * 0.5
M = test_df["product"].map(hi).fillna(train_df["y"].quantile(0.99)) * 2.0
y_pred_test = np.clip(y_pred_test, m.to_numpy(), M.to_numpy())

# 9) Metrics & per-product breakdown
print("\n=== STAPLES TEST METRICS ===")
print(f"MAE : {mean_absolute_error(y_test_true, y_pred_test):,.3f}")
print(f"RMSE: {rmse(y_test_true, y_pred_test):,.3f}")
print(f"sMAPE: {smape(y_test_true, y_pred_test):.2f}%")

# attach predictions
out_df = test_df.copy()
out_df["y_pred"] = y_pred_test

def metrics_for_group(g: pd.DataFrame) -> pd.Series:
    gg = g.loc[g["y_pred"].notna() & g["y"].notna(), ["y", "y_pred"]]
    n = int(len(gg))
    if n == 0:
        return pd.Series({"sMAPE": np.nan, "MAE": np.nan, "RMSE": np.nan, "N": 0})
    mae  = float(np.abs(gg["y"] - gg["y_pred"]).mean())
    rmse_v = float(np.sqrt(((gg["y"] - gg["y_pred"])**2).mean()))
    denom = (np.abs(gg["y"]) + np.abs(gg["y_pred"]) + 1e-9) / 2.0
    sm   = float((np.abs(gg["y_pred"] - gg["y"]) / denom).mean() * 100.0)
    return pd.Series({"sMAPE": sm, "MAE": mae, "RMSE": rmse_v, "N": n})

by_prod = (out_df.groupby("product", observed=False)
                 .apply(metrics_for_group)
                 .reset_index()
                 .sort_values("sMAPE", ascending=False))

print("\nWorst staples by sMAPE:")
print(by_prod.head(10).to_string(index=False))

/var/folders/z1/szmsj4mj2_jgj9dx7p3cz92w0000gn/T/ipykernel_29268/2457992708.py:96: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  pd.api.types.is_categorical_dtype(out[c]) or
/var/folders/z1/szmsj4mj2_jgj9dx7p3cz92w0000gn/T/ipykernel_29268/2457992708.py:92: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(out[c]):
/var/folders/z1/szmsj4mj2_jgj9dx7p3cz92w0000gn/T/ipykernel_29268/2457992708.py:92: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(out[c]):
/var/folders/z1/szmsj4mj2_jgj9dx7p3cz92w0000gn/T/ipykernel_29268/2457992708.py:96: DeprecationWarning: is_categorical_dtype is deprecated and will be removed i

Tuning staples model with 40 trials…


[I 2025-10-25 12:29:37,166] Trial 0 finished with value: 1.883623362151949 and parameters: {'n_estimators': 1162, 'learning_rate': 0.015683503930815343, 'max_depth': 5, 'min_child_weight': 6.771436939990866, 'subsample': 0.8017806293002239, 'colsample_bytree': 0.8738505559082057, 'gamma': 1.2624277917206517, 'reg_alpha': 0.0001254085050201393, 'reg_lambda': 0.0011405480670603888}. Best is trial 0 with value: 1.883623362151949.
[I 2025-10-25 12:29:38,103] Trial 1 finished with value: 2.4424633156803943 and parameters: {'n_estimators': 670, 'learning_rate': 0.21083476708743015, 'max_depth': 5, 'min_child_weight': 0.17063720795201975, 'subsample': 0.7960110908234259, 'colsample_bytree': 0.5672388230881045, 'gamma': 2.341630767169926, 'reg_alpha': 0.03434704104250717, 'reg_lambda': 2.0065401868252615e-08}. Best is trial 0 with value: 1.883623362151949.
[I 2025-10-25 12:29:39,850] Trial 2 finished with value: 2.6928124047718223 and parameters: {'n_estimators': 1062, 'learning_rate': 0.17461

Best sMAPE: 1.7865611823371488
Best params: {'n_estimators': 1999, 'learning_rate': 0.03445348757177523, 'max_depth': 11, 'min_child_weight': 1.0705260182528338, 'subsample': 0.7926410897298273, 'colsample_bytree': 0.7273866944456198, 'gamma': 0.15093154465589476, 'reg_alpha': 0.0001681070059026855, 'reg_lambda': 0.11063442723654303}

=== STAPLES TEST METRICS ===
MAE : 12.274
RMSE: 29.607
sMAPE: 7.57%

Worst staples by sMAPE:
               product     sMAPE       MAE      RMSE    N
 Refined Vegetable Oil 28.391415 79.454699 87.775995 35.0
         Sorghum (Red)  7.548771  4.350891  6.525895 32.0
       Sorghum (White)  6.597296  4.340350  7.546356 26.0
   Maize Grain (White)  5.499172  2.690474  4.021647 56.0
            Mixed Teff  5.351611  6.647818  7.193303 24.0
         Rice (Milled)  5.057807  6.475950 10.780830 35.0
           Wheat Flour  4.356458  4.394201  7.740979 35.0
           Wheat Grain  3.599267  2.599030  3.600205 56.0
         Refined sugar  2.712448  3.609062  5.57

/var/folders/z1/szmsj4mj2_jgj9dx7p3cz92w0000gn/T/ipykernel_29268/2457992708.py:314: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(metrics_for_group)


In [21]:
# ======================= train_and_export.py =======================
# Trains the staples-only model, exports model + metadata,
# and produces multi-month recursive forecasts for Streamlit.
# ================================================================

import numpy as np
import pandas as pd
from pathlib import Path
import warnings
import joblib
from datetime import datetime

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import optuna
from xgboost import XGBRegressor

# ----------------- Config -----------------
PARQUET_PATH = "/Users/nataschajademinnitt/Documents/5_data/food_security/etl/ethiopia_foodprices_model_panel.parquet"
OUTPUT_DIR    = Path("/Users/nataschajademinnitt/Documents/5_data/food_security/etl/artifacts")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

TEST_HORIZON_MONTHS = 6             # last H months = test
OPTUNA_TRIALS       = 40
FORECAST_HORIZON    = 6             # months to predict into the future (can increase)

# ----------------- Helpers -----------------
def month_start(s: pd.Series) -> pd.Series:
    return pd.to_datetime(s, errors="coerce").dt.to_period("M").dt.to_timestamp()

def smape(y_true, y_pred, eps=1e-9) -> float:
    y_true = np.asarray(y_true, float)
    y_pred = np.asarray(y_pred, float)
    denom = (np.abs(y_true) + np.abs(y_pred) + eps) / 2.0
    return float(np.mean(np.abs(y_pred - y_true) / denom) * 100.0)

def rmse(y_true, y_pred) -> float:
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

KEY_CATS = {"admin_1", "product"}
NUMERIC_LIKELY = {
    "value_imputed","value_mean","value_median","value_orig","n_obs",
    "rfh_month","rfh_avg_month","rfq_month","rain_anom_pct",
    "fao_category_index","fao_food_price_index",
    "ptm_severity","population_2023",
    "y","y_lag1","y_lag3","y_lag6","y_lag12",
    "roll_mean_3","roll_std_3","roll_mean_12",
    "month_num","mo_sin","mo_cos",
}

def decategorize_numerics(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in out.columns:
        if c in KEY_CATS:
            if not pd.api.types.is_categorical_dtype(out[c]):
                out[c] = out[c].astype("category")
            continue
        if (c in NUMERIC_LIKELY or
            pd.api.types.is_categorical_dtype(out[c]) or
            pd.api.types.is_object_dtype(out[c])):
            out[c] = pd.to_numeric(out[c], errors="coerce")
    return out

def ensure_cols(df: pd.DataFrame, cols: list[str], dtype="float64") -> pd.DataFrame:
    out = df.copy()
    for c in cols:
        if c not in out.columns:
            out[c] = np.nan
        if dtype is not None:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                try:
                    out[c] = out[c].astype(dtype)
                except Exception:
                    pass
    return out

def fit_xgb_compat(params, X_tr, y_tr, X_val=None, y_val=None, early_rounds=100):
    # Try newer API (>=2.0)
    try:
        from xgboost.callback import EarlyStopping
        mdl = XGBRegressor(**params)
        if X_val is not None:
            mdl.fit(X_tr, y_tr, eval_set=[(X_val, y_val)],
                    callbacks=[EarlyStopping(rounds=early_rounds, save_best=True)])
        else:
            mdl.fit(X_tr, y_tr)
        return mdl
    except (TypeError, ImportError):
        pass
    # Fallback older
    try:
        mdl = XGBRegressor(**{k:v for k,v in params.items() if k!="eval_metric"})
        if X_val is not None:
            mdl.fit(X_tr, y_tr, eval_set=[(X_val, y_val)],
                    eval_metric=params.get("eval_metric","rmse"),
                    early_stopping_rounds=early_rounds)
        else:
            mdl.fit(X_tr, y_tr)
        return mdl
    except TypeError:
        pass
    # Last
    mdl = XGBRegressor(**{k:v for k,v in params.items() if k!="eval_metric"})
    mdl.fit(X_tr, y_tr)
    return mdl

def impute_features(df_in: pd.DataFrame) -> pd.DataFrame:
    df = decategorize_numerics(df_in)
    df["month"] = month_start(df["month"])

    rain_cols = ["rfh_month","rfh_avg_month","rfq_month","rain_anom_pct"]
    fao_cols  = ["fao_category_index","fao_food_price_index"]
    df = ensure_cols(df, rain_cols + fao_cols)

    # flags
    for c in rain_cols:
        df[f"{c}_was_missing"] = df[c].isna().astype("int8")

    # recompute anomaly where possible
    can = df["rfh_month"].notna() & df["rfh_avg_month"].gt(0)
    df.loc[can,"rain_anom_pct"] = 100.0*(df.loc[can,"rfh_month"]/df.loc[can,"rfh_avg_month"] - 1.0)

    # same-month median fill
    for c in rain_cols + fao_cols:
        df[c] = df[c].fillna(df.groupby("month", observed=False)[c].transform("median"))

    # gentle past-only rolling fill by admin
    df = df.sort_values(["admin_1","month"])
    for c in rain_cols:
        df[c] = (df.groupby("admin_1", group_keys=False)[c]
                   .apply(lambda s: s.fillna(s.shift(1).rolling(3, min_periods=1).mean())))

    # ptm & population
    if "ptm_severity" in df.columns:
        df["ptm_missing"] = df["ptm_severity"].isna().astype("int8")
        df["ptm_severity"] = df["ptm_severity"].fillna(0).astype("int8")
    else:
        df["ptm_missing"] = 1; df["ptm_severity"] = 0

    if "population_2023" in df.columns:
        df["pop_missing"] = df["population_2023"].isna().astype("int8")
        adm_med = df.groupby("admin_1", observed=False)["population_2023"].transform("median")
        df["population_2023"] = df["population_2023"].fillna(adm_med)
        df["population_2023"] = df["population_2023"].fillna(df["population_2023"].median())
    else:
        df["pop_missing"] = 1; df["population_2023"] = np.nan

    # FAO forward-fill by time
    df = df.sort_values(["month"])
    for c in fao_cols:
        df[c] = df[c].ffill()

    return df

def build_features(df_in: pd.DataFrame) -> pd.DataFrame:
    df = df_in.sort_values(["admin_1","product","month"]).copy()
    df["y"] = df["value_imputed"].astype("float32")

    g = df.groupby(["admin_1","product"], group_keys=False)["y"]
    df["y_lag1"]  = g.shift(1)
    df["y_lag3"]  = g.shift(3)
    df["y_lag6"]  = g.shift(6)
    df["y_lag12"] = g.shift(12)

    df["roll_mean_3"]  = g.apply(lambda s: s.shift(1).rolling(3,  min_periods=1).mean())
    df["roll_std_3"]   = g.apply(lambda s: s.shift(1).rolling(3,  min_periods=2).std())
    df["roll_mean_12"] = g.apply(lambda s: s.shift(1).rolling(12, min_periods=3).mean())

    df["month_num"] = df["month"].dt.month
    df["mo_sin"] = np.sin(2*np.pi*df["month_num"]/12.0)
    df["mo_cos"] = np.cos(2*np.pi*df["month_num"]/12.0)

    # minimal history
    df = df[df["y_lag1"].notna()].copy()
    return df

def encode_ids(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in ["admin_1","product"]:
        out[c] = out[c].astype("category")
        out[f"{c}_code"] = out[c].cat.codes.astype("int32")
    return out

def pick_features(df: pd.DataFrame) -> list[str]:
    drop = {
        "y","month","admin_1","product",
        "value_imputed","value_median","value_mean","value_orig",
        "n_obs","impute_method","sources","period_date"
    }
    return [c for c in df.columns if c not in drop]

def time_split(df: pd.DataFrame, horizon=6):
    df = df.sort_values(["month","admin_1","product"]).reset_index(drop=True)
    last_m = df["month"].max().to_period("M").to_timestamp()
    cut_m  = (last_m.to_period("M") - horizon).to_timestamp()
    return df[df["month"] <= cut_m].copy(), df[df["month"] > cut_m].copy()

def tune_xgb(X_train, y_train, n_trials=40, seed=42):
    tscv = TimeSeriesSplit(n_splits=5)
    def objective(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 300, 2000),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "min_child_weight": trial.suggest_float("min_child_weight", 1e-2, 10.0, log=True),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "gamma": trial.suggest_float("gamma", 0.0, 5.0),
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
            "tree_method": "hist", "random_state": seed, "n_jobs": -1, "eval_metric": "rmse",
        }
        smapes = []
        for tr_idx, va_idx in tscv.split(X_train):
            X_tr, X_val = X_train.iloc[tr_idx], X_train.iloc[va_idx]
            y_tr, y_val = y_train[tr_idx],    y_train[va_idx]
            mdl = fit_xgb_compat(params, X_tr, y_tr, X_val, y_val, early_rounds=100)
            smapes.append(smape(y_val, mdl.predict(X_val)))
            trial.report(float(np.mean(smapes)), step=len(smapes))
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()
        return float(np.mean(smapes))

    study = optuna.create_study(direction="minimize", study_name="xgb-staples-smape")
    study.optimize(objective, n_trials=n_trials, show_progress_bar=False)
    print("Best sMAPE:", study.best_value)
    print("Best params:", study.best_trial.params)
    best = study.best_trial.params
    best.update({"tree_method":"hist","random_state":seed,"n_jobs":-1,"eval_metric":"rmse"})
    return best

# -------------- Recursive forecast builder --------------
def future_months(start_month: pd.Timestamp, horizon: int) -> pd.DatetimeIndex:
    start_m = start_month.to_period("M")
    rng = pd.period_range(start=start_m + 1, periods=horizon, freq="M").to_timestamp()
    return pd.DatetimeIndex(rng)

def recursive_forecast(model: XGBRegressor,
                       last_hist: pd.DataFrame,
                       feats: list[str],
                       horizon: int = 6) -> pd.DataFrame:
    """
    last_hist: DataFrame with columns ['admin_1','product','month','y', lag/roll features...]
               containing at least the last 12 months per series to compute features.
    Returns forecast rows for next `horizon` months with columns:
      admin_1, product, month, y_pred
    """
    out_rows = []
    # Work per series to maintain its own lag/rolling state
    for (adm, prod), g in last_hist.groupby(["admin_1","product"], sort=False):
        g = g.sort_values("month").copy()
        # Use the last available month to start
        last_m = g["month"].max()
        future_idx = future_months(last_m, horizon)
        # We’ll maintain a small working frame with just needed cols
        work = g[["month","y"]].copy()
        # Precompute month_num→sin/cos for all future months
        mo_num = future_idx.month
        mo_sin = np.sin(2*np.pi*mo_num/12.0)
        mo_cos = np.cos(2*np.pi*mo_num/12.0)

        # Values for exogenous features in the future:
        # simple, robust choice = keep last known per series (you can replace with true future drivers if available)
        exo_cols = [c for c in feats if c not in {"admin_1_code","product_code",
                                                  "y_lag1","y_lag3","y_lag6","y_lag12",
                                                  "roll_mean_3","roll_std_3","roll_mean_12",
                                                  "month_num","mo_sin","mo_cos"}]
        exo_defaults = {}
        if exo_cols:
            last_exo = g.iloc[-1:].reindex(columns=exo_cols).to_dict(orient="records")[0]
            exo_defaults = {k: (0.0 if pd.isna(v) else float(v)) for k, v in last_exo.items()}

        # Codes (fixed)
        admin_code = int(g["admin_1_code"].iloc[-1]) if "admin_1_code" in g.columns else 0
        product_code = int(g["product_code"].iloc[-1]) if "product_code" in g.columns else 0

        # Iterate months
        for i, (m, s, c) in enumerate(zip(future_idx, mo_sin, mo_cos)):
            # construct feature row using *past-only* info
            # compute lags from `work`
            y_lag1  = work["y"].iloc[-1] if len(work) >= 1 else np.nan
            y_lag3  = work["y"].iloc[-3] if len(work) >= 3 else np.nan
            y_lag6  = work["y"].iloc[-6] if len(work) >= 6 else np.nan
            y_lag12 = work["y"].iloc[-12] if len(work) >= 12 else np.nan

            roll_mean_3  = work["y"].shift(1).rolling(3,  min_periods=1).mean().iloc[-1] if len(work) >= 1 else np.nan
            roll_std_3   = work["y"].shift(1).rolling(3,  min_periods=2).std().iloc[-1]  if len(work) >= 2 else np.nan
            roll_mean_12 = work["y"].shift(1).rolling(12, min_periods=3).mean().iloc[-1] if len(work) >= 3 else np.nan

            month_num = m.month

            feat_row = {
                "admin_1_code": admin_code,
                "product_code": product_code,
                "y_lag1": y_lag1, "y_lag3": y_lag3, "y_lag6": y_lag6, "y_lag12": y_lag12,
                "roll_mean_3": roll_mean_3, "roll_std_3": roll_std_3, "roll_mean_12": roll_mean_12,
                "month_num": month_num, "mo_sin": s, "mo_cos": c,
            }
            # add exogenous defaults
            feat_row.update(exo_defaults)

            # Align to model features (missing → NaN -> model can handle)
            X_f = pd.DataFrame([feat_row])[feats].apply(pd.to_numeric, errors="coerce")
            # Predict (remember model was trained on log1p(y))
            y_hat = float(np.expm1(model.predict(X_f)[0]))
            out_rows.append({"admin_1": adm, "product": prod, "month": m, "y_pred": y_hat})

            # append to work to update lags for next step
            work = pd.concat([work, pd.DataFrame({"month":[m], "y":[y_hat]})], ignore_index=True)

    return pd.DataFrame(out_rows)

# ============================== TRAIN & EXPORT ===============================
if __name__ == "__main__":
    # 1) Load
    panel = pd.read_parquet(PARQUET_PATH)
    assert {"admin_1","product","month","value_imputed"}.issubset(panel.columns)

    # de-categorize once
    panel = decategorize_numerics(panel)
    # force any non-key categoricals to numeric
    for c in panel.columns:
        if pd.api.types.is_categorical_dtype(panel[c]) and c not in KEY_CATS:
            panel[c] = pd.to_numeric(panel[c].astype(object), errors="coerce")

    # 2) Drop livestock
    LIVESTOCK = {
        "Goats (Local Quality)","Sheep (Local Quality)",
        "Camels (Local Quality)","Oxen (Local Quality)"
    }
    staples = panel[~panel["product"].isin(LIVESTOCK)].copy()

    # 3) Features
    staples = impute_features(staples)
    df_feat  = build_features(staples)
    df_feat  = encode_ids(df_feat)
    df_feat  = df_feat.sort_values(["month","admin_1","product"]).reset_index(drop=True)
    feats = [c for c in df_feat.columns if c not in {
        "y","month","admin_1","product",
        "value_imputed","value_median","value_mean","value_orig",
        "n_obs","impute_method","sources","period_date"
    }]
    df_feat[feats] = df_feat[feats].apply(pd.to_numeric, errors="coerce").astype("float32")

    # 4) Split
    train_df = df_feat[df_feat["month"] <= (df_feat["month"].max().to_period("M") - TEST_HORIZON_MONTHS).to_timestamp()].copy()
    test_df  = df_feat[df_feat["month"]  > (df_feat["month"].max().to_period("M") - TEST_HORIZON_MONTHS).to_timestamp()].copy()
    X_train, X_test = train_df[feats], test_df[feats]
    y_train_log = np.log1p(train_df["y"].to_numpy())
    y_test_true = test_df["y"].to_numpy()

    # 5) Tune
    print(f"[{datetime.now()}] Tuning ({OPTUNA_TRIALS} trials)…")
    best_params = tune_xgb(X_train, y_train_log, n_trials=OPTUNA_TRIALS, seed=42)

    # 6) Early stopping on tail of TRAIN
    last_train_m = train_df["month"].max()
    val_cut = (last_train_m.to_period("M") - 2).to_timestamp()  # last 3 months
    val_mask = train_df["month"] >= val_cut
    X_tr_es, y_tr_es = X_train[~val_mask], y_train_log[~val_mask]
    X_va_es, y_va_es = X_train[val_mask],  y_train_log[val_mask]
    if len(X_va_es) == 0 or len(X_tr_es) == 0:
        X_tr_es, y_tr_es = X_train, y_train_log
        X_va_es = y_va_es = None

    # 7) Fit final & predict test
    final_model = fit_xgb_compat(best_params, X_tr_es, y_tr_es, X_va_es, y_va_es, early_rounds=200)
    y_pred_test = np.expm1(final_model.predict(X_test))

    print("\n=== STAPLES TEST METRICS ===")
    print(f"MAE : {np.mean(np.abs(y_test_true - y_pred_test)):,.3f}")
    print(f"RMSE: {rmse(y_test_true, y_pred_test):,.3f}")
    print(f"sMAPE: {smape(y_test_true, y_pred_test):.2f}%")

    # 8) Save artifacts
    model_path   = OUTPUT_DIR / "xgb_staples_model.joblib"
    feats_path   = OUTPUT_DIR / "xgb_staples_features.json"
    last_hist_pq = OUTPUT_DIR / "xgb_staples_last_history.parquet"
    test_pred_pq = OUTPUT_DIR / "xgb_staples_test_predictions.parquet"

    joblib.dump(final_model, model_path)
    pd.Series(feats, name="features").to_json(feats_path, orient="values")

    test_out = test_df[["admin_1","product","month","y"]].copy()
    test_out["y_pred"] = y_pred_test
    test_out.to_parquet(test_pred_pq, index=False)

    # 9) Build a compact "last history" frame for recursive forecasting
    # Keep last 15 months per series to compute lags/rolls safely
    last_blocks = (
        df_feat.sort_values(["admin_1","product","month"])
              .groupby(["admin_1","product"], as_index=False)
              .tail(15)
              .reset_index(drop=True)
    )
    last_blocks.to_parquet(last_hist_pq, index=False)

    # 10) Generate forward forecasts now (optional: or do it in Streamlit)
    print(f"\n[{datetime.now()}] Building {FORECAST_HORIZON}-month recursive forecasts…")
    # Use the same features list `feats`
    future_df = recursive_forecast(final_model, last_blocks, feats, horizon=FORECAST_HORIZON)
    future_path = OUTPUT_DIR / "xgb_staples_future_forecast.parquet"
    future_df.to_parquet(future_path, index=False)

    print("\nSaved:")
    print("  Model           ->", model_path)
    print("  Feature list    ->", feats_path)
    print("  Last history    ->", last_hist_pq)
    print("  Test predictions->", test_pred_pq)
    print("  Future forecasts->", future_path)


/var/folders/z1/szmsj4mj2_jgj9dx7p3cz92w0000gn/T/ipykernel_29268/1019645448.py:59: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  pd.api.types.is_categorical_dtype(out[c]) or
/var/folders/z1/szmsj4mj2_jgj9dx7p3cz92w0000gn/T/ipykernel_29268/1019645448.py:55: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(out[c]):
/var/folders/z1/szmsj4mj2_jgj9dx7p3cz92w0000gn/T/ipykernel_29268/1019645448.py:55: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(out[c]):
/var/folders/z1/szmsj4mj2_jgj9dx7p3cz92w0000gn/T/ipykernel_29268/1019645448.py:59: DeprecationWarning: is_categorical_dtype is deprecated and will be removed i

[2025-10-25 12:34:20.465786] Tuning (40 trials)…


[I 2025-10-25 12:34:23,059] Trial 0 finished with value: 2.5909003967349946 and parameters: {'n_estimators': 1610, 'learning_rate': 0.276206748644435, 'max_depth': 7, 'min_child_weight': 0.7001440005267261, 'subsample': 0.7064562857829264, 'colsample_bytree': 0.6412959759197572, 'gamma': 2.9111580514058715, 'reg_alpha': 0.00019003726177874546, 'reg_lambda': 0.20112727332356684}. Best is trial 0 with value: 2.5909003967349946.
[I 2025-10-25 12:34:27,866] Trial 1 finished with value: 2.144116314439753 and parameters: {'n_estimators': 1928, 'learning_rate': 0.01667350193709267, 'max_depth': 10, 'min_child_weight': 0.03767687709154869, 'subsample': 0.9751628442076601, 'colsample_bytree': 0.9258384879077287, 'gamma': 4.479894224603827, 'reg_alpha': 0.00018975561123092013, 'reg_lambda': 9.731188034963155e-05}. Best is trial 1 with value: 2.144116314439753.
[I 2025-10-25 12:34:29,351] Trial 2 finished with value: 2.0730644744538704 and parameters: {'n_estimators': 921, 'learning_rate': 0.0300

Best sMAPE: 1.7972009496258785
Best params: {'n_estimators': 867, 'learning_rate': 0.01025557781974342, 'max_depth': 10, 'min_child_weight': 1.1141452098165265, 'subsample': 0.682444539742094, 'colsample_bytree': 0.8153022194774273, 'gamma': 0.400828486905724, 'reg_alpha': 0.0007415513625505704, 'reg_lambda': 9.74453754947614e-07}

=== STAPLES TEST METRICS ===
MAE : 10.155
RMSE: 23.676
sMAPE: 6.57%

[2025-10-25 12:35:27.607455] Building 6-month recursive forecasts…


/var/folders/z1/szmsj4mj2_jgj9dx7p3cz92w0000gn/T/ipykernel_29268/1019645448.py:393: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["admin_1","product"], as_index=False)
/var/folders/z1/szmsj4mj2_jgj9dx7p3cz92w0000gn/T/ipykernel_29268/1019645448.py:251: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (adm, prod), g in last_hist.groupby(["admin_1","product"], sort=False):



Saved:
  Model           -> /Users/nataschajademinnitt/Documents/5_data/food_security/etl/artifacts/xgb_staples_model.joblib
  Feature list    -> /Users/nataschajademinnitt/Documents/5_data/food_security/etl/artifacts/xgb_staples_features.json
  Last history    -> /Users/nataschajademinnitt/Documents/5_data/food_security/etl/artifacts/xgb_staples_last_history.parquet
  Test predictions-> /Users/nataschajademinnitt/Documents/5_data/food_security/etl/artifacts/xgb_staples_test_predictions.parquet
  Future forecasts-> /Users/nataschajademinnitt/Documents/5_data/food_security/etl/artifacts/xgb_staples_future_forecast.parquet
